# GPT 2 Implementation

In [1]:
# read in the text line by line
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# establish the characters and the vocabulary size
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [2]:
# initialise lookuptables
char_ind = {}
ind_char = {}
for i, ch in enumerate(chars):
    char_ind[ch] = i
    ind_char[i] = ch

In [9]:
# encoder and decoder functions
def encode(input_str:str) -> list:
    return [char_ind[s] for s in input_str]

def decode(input_list:list) -> str:
    return ''.join([ind_char[l] for l in input_list])